In [10]:
import pandas as pd

# Load the datasets
survey_data = pd.read_csv('adjusted_reponse_survey.csv')
contract_data = pd.read_csv('Notebooks\experiment\curr_data.csv')

from scipy.stats import wilcoxon, shapiro

# Merge the datasets on player names for analysis
merged_data = contract_data.merge(
    survey_data, how='left', left_on='Player ID', right_on='PLAYER NAME'
)

contract_data.head()


<>:5: SyntaxWarning: invalid escape sequence '\e'
<>:5: SyntaxWarning: invalid escape sequence '\e'
C:\Users\telat\AppData\Local\Temp\ipykernel_59028\839069531.py:5: SyntaxWarning: invalid escape sequence '\e'
  contract_data = pd.read_csv('Notebooks\experiment\curr_data.csv')


,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Realized Retailer Profit,Predicted Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,23.45,234.6,300,0.561167,77.72,5.28,268.64,415,240,0.607045
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,15.35,347.8,320,0.479185,82.80,8.20,356.60,364,400,0.505135
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,15.35,347.8,320,0.479185,82.80,8.20,356.60,364,124,0.505135
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,24.00,153.0,306,0.666667,72.00,3.00,189.00,450,153,0.704225
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,23.45,234.6,300,0.561167,77.72,5.28,268.64,415,240,0.607045


In [20]:
# Standardize identifiers in both datasets to facilitate merging

top_avg_riskaverse = survey_data.sort_values(by='Avg_RiskAverseCoeff').head(4)
bottom_avg_riskaverse = survey_data.sort_values(by='Avg_RiskAverseCoeff').tail(4)

# Extracting 'PLAYER NAME' values
top_avg_riskaverse_players = top_avg_riskaverse['PLAYER NAME'].tolist()
bottom_avg_riskaverse_players = bottom_avg_riskaverse['PLAYER NAME'].tolist()

# Filtering corresponding data in curr_data based on 'Player ID'
top_avg_riskaverse_data = contract_data[contract_data['Player ID'].isin(top_avg_riskaverse_players)]
bottom_avg_riskaverse_data = contract_data[contract_data['Player ID'].isin(bottom_avg_riskaverse_players)]

top_avg_riskaverse_summary = top_avg_riskaverse_data[['Player ID', 'Stock', 'Under/Overstock_wrt_Optimal']].describe()
bottom_avg_riskaverse_summary = bottom_avg_riskaverse_data[['Player ID', 'Stock', 'Under/Overstock_wrt_Optimal']].describe()

top_avg_riskaverse_summary, bottom_avg_riskaverse_summary



(            Stock  Under/Overstock_wrt_Optimal
 count  160.000000                   160.000000
 mean    95.037500                     8.768750
 std     42.447435                    35.504667
 min      0.000000                   -91.000000
 25%     70.750000                    -8.500000
 50%    100.000000                    17.000000
 75%    135.000000                    35.000000
 max    150.000000                    67.000000,
             Stock  Under/Overstock_wrt_Optimal
 count  160.000000                   160.000000
 mean    85.381250                    -1.056250
 std     41.032718                    37.307366
 min      0.000000                   -91.000000
 25%     60.750000                   -26.500000
 50%     90.000000                     5.000000
 75%    115.000000                    29.000000
 max    150.000000                    59.000000)

In [23]:
from scipy.stats import ttest_ind, mannwhitneyu

# Hypothesis 1: Does Risk Aversion affect the difference between actual and optimal stock levels?
# Null Hypothesis (H0): There is no significant difference in the stock-keeping behavior of high-risk-averse and low-risk-averse individuals.
# Alternative Hypothesis (H1): There is a significant difference in the stock-keeping behavior based on risk aversion.

# Assuming the data for 'top_avg_riskaverse_data' and 'bottom_avg_riskaverse_data' has been successfully filtered

# Extracting the 'Under/Overstock_wrt_Optimal' values for the two groups
top_avg_deviation = top_avg_riskaverse_data['Under/Overstock_wrt_Optimal'].dropna()
bottom_avg_deviation = bottom_avg_riskaverse_data['Under/Overstock_wrt_Optimal'].dropna()

# Performing a t-test to compare means if data is assumed normally distributed
# If not normally distributed, we can use the Mann-Whitney U test

t_stat, p_value = mannwhitneyu(top_avg_deviation.dropna(), bottom_avg_deviation.dropna(), alternative='greater')
test_result = f"T-test results for Avg_RiskAverseCoeff groups - t-statistic: {t_stat}, p-value: {p_value}"


test_result


'T-test results for Avg_RiskAverseCoeff groups - t-statistic: 15035.0, p-value: 0.003420840430089309'

In [29]:
# Hypothesis Test: Impact of Self-Esteem on Deviation from Optimal Stock Levels
# Null Hypothesis (H0): There is no significant difference in the deviation from optimal stock level between high and low self-esteem groups.
# Alternative Hypothesis (H1): There is a significant difference in deviation based on self-esteem.

# Step 1: Sorting by 'Avg_SelfEsteem' and selecting top and bottom 4
top_self_esteem = survey_data.sort_values(by='Diff_SelfEsteem').tail(4)
bottom_self_esteem = survey_data.sort_values(by='Diff_SelfEsteem').head(4)

# Extracting 'PLAYER NAME' values
top_self_esteem_players = top_self_esteem['PLAYER NAME'].tolist()
bottom_self_esteem_players = bottom_self_esteem['PLAYER NAME'].tolist()

# Filtering corresponding data in curr_data based on 'Player ID'
top_self_esteem_data = contract_data[contract_data['Player ID'].isin(top_self_esteem_players)]
bottom_self_esteem_data = contract_data[contract_data['Player ID'].isin(bottom_self_esteem_players)]

# Extracting 'Under/Overstock_wrt_Optimal' values for the two groups
top_self_esteem_deviation = top_self_esteem_data['Under/Overstock_wrt_Optimal'].dropna()
bottom_self_esteem_deviation = bottom_self_esteem_data['Under/Overstock_wrt_Optimal'].dropna()

# Performing a t-test to compare the means between the two groups

t_stat_self_esteem, p_value_self_esteem = mannwhitneyu(top_self_esteem_deviation.dropna(), bottom_self_esteem_deviation.dropna(), alternative='greater')
self_esteem_test_result = f"T-test results for Self-Esteem groups - t-statistic: {t_stat_self_esteem}, p-value: {p_value_self_esteem}"


self_esteem_test_result


#The Mann-Whitney U test yields a p-value of approximately 0.0036, indicating a statistically significant 
#difference in deviations from optimal stock levels between high and low self-esteem groups. 
#This suggests that self-esteem may indeed influence ordering behavior, with higher 
#self-esteem potentially aligning more closely with optimal stock levels.



'T-test results for Self-Esteem groups - t-statistic: 15418.5, p-value: 0.0007740541654231287'

In [32]:
# Hypothesis Test: Influence of Fairness Index on Wholesale Price Acceptance
# Null Hypothesis (H0): There is no significant difference in the reaction to wholesale price changes based on fairness index.
# Alternative Hypothesis (H1): There is a significant difference in wholesale price acceptance based on fairness.

# Step 1: Sorting by 'Fairness_Interaction' and selecting top and bottom 4
top_fairness = survey_data.sort_values(by='Fairness_Interaction').tail(4)
bottom_fairness = survey_data.sort_values(by='Fairness_Interaction').head(4)

# Extracting 'PLAYER NAME' values
top_fairness_players = top_fairness['PLAYER NAME'].tolist()
bottom_fairness_players = bottom_fairness['PLAYER NAME'].tolist()

# Filtering corresponding data in curr_data based on 'Player ID'
top_fairness_data = contract_data[contract_data['Player ID'].isin(top_fairness_players)]
bottom_fairness_data = contract_data[contract_data['Player ID'].isin(bottom_fairness_players)]

# Assuming 'Wholesale p.' is the column indicating wholesale price reaction or acceptance
top_fairness_wholesale = top_fairness_data['Wholesale p.'].dropna()
bottom_fairness_wholesale = bottom_fairness_data['Wholesale p.'].dropna()

# Performing a t-test to compare means between the two groups

t_stat_fairness, p_value_fairness = mannwhitneyu(top_fairness_wholesale.dropna(), bottom_fairness_wholesale.dropna(), alternative='greater')
fairness_test_result = f"T-test results for Fairness Interaction groups - t-statistic: {t_stat_fairness}, p-value: {p_value_fairness}"


fairness_test_result



#The Mann-Whitney U test results in a p-value of approximately 0.0002, indicating a statistically significant difference 
#in reactions to wholesale price changes based on fairness index. This suggests that individuals with higher fairness indexes
 #may respond differently to price fluctuations compared to those with lower fairness indexes, potentially being more 
 #resistant to or influenced by changes in wholesale prices.



'T-test results for Fairness Interaction groups - t-statistic: 15471.0, p-value: 0.0003935102935428898'

In [ ]:
# Hypothesis Test: Regret Sensitivity and Over/Under-Stocking Behavior
# Null Hypothesis (H0): There is no significant difference in over/under-stocking behavior based on regret sensitivity.
# Alternative Hypothesis (H1): There is a significant difference in over/under-stocking based on regret sensitivity.

# Step 1: Sorting by 'Avg_RegretScale' and selecting top and bottom 4
top_regret = survey_data.sort_values(by='Avg_RegretScale').tail(4)
bottom_regret = survey_data.sort_values(by='Avg_RegretScale').head(4)

# Extracting 'PLAYER NAME' values
top_regret_players = top_regret['PLAYER NAME'].tolist()
bottom_regret_players = bottom_regret['PLAYER NAME'].tolist()

# Filtering corresponding data in curr_data based on 'Player ID'
top_regret_data = contract_data[contract_data['Player ID'].isin(top_regret_players)]
bottom_regret_data = contract_data[contract_data['Player ID'].isin(bottom_regret_players)]

# Extracting 'Under/Overstock_wrt_Optimal' values for the two groups
top_regret_deviation = top_regret_data['Under/Overstock_wrt_Optimal'].dropna()
bottom_regret_deviation = bottom_regret_data['Under/Overstock_wrt_Optimal'].dropna()

# Performing a t-test to compare means between the two groups

t_stat_regret, p_value_regret = mannwhitneyu(top_regret_deviation.dropna(), bottom_regret_deviation.dropna(), alternative='less')
regret_test_result = f"T-test results for Regret Sensitivity groups - t-statistic: {t_stat_regret}, p-value: {p_value_regret}"


regret_test_result




'T-test results for Regret Sensitivity groups - t-statistic: 8029.5, p-value: 4.012201934154288e-09'

In [36]:
# Hypothesis Test: Fairness Index and Retailer Profit Share
# Null Hypothesis (H0): There is no significant difference in retailer profit share based on fairness index.
# Alternative Hypothesis (H1): There is a significant difference in retailer profit share based on fairness index.

# Step 1: Sorting by 'Diff_FairnessIndex' and selecting top and bottom 4
top_fairness_diff = survey_data.sort_values(by='Diff_FairnessIndex').tail(4)
bottom_fairness_diff = survey_data.sort_values(by='Diff_FairnessIndex').head(4)

# Extracting 'PLAYER NAME' values
top_fairness_diff_players = top_fairness_diff['PLAYER NAME'].tolist()
bottom_fairness_diff_players = bottom_fairness_diff['PLAYER NAME'].tolist()

# Filtering corresponding data in curr_data based on 'Player ID'
top_fairness_diff_data = contract_data[contract_data['Player ID'].isin(top_fairness_diff_players)]
bottom_fairness_diff_data = contract_data[contract_data['Player ID'].isin(bottom_fairness_diff_players)]

# Extracting 'Realized Retailer Profit' values for the two groups as a proxy for retailer profit share
top_fairness_diff_profit = top_fairness_diff_data['Realized Retailer Profit'] / (top_fairness_diff_data['Realized Retailer Profit'] + top_fairness_diff_data['Realized_Mfg_Profit'])
bottom_fairness_diff_profit = bottom_fairness_diff_data['Realized Retailer Profit'] / (bottom_fairness_diff_data['Realized Retailer Profit'] + bottom_fairness_diff_data['Realized_Mfg_Profit'])

# Performing a t-test to compare means between the two groups

t_stat_fairness_diff, p_value_fairness_diff = mannwhitneyu(top_fairness_diff_profit.dropna(), bottom_fairness_diff_profit.dropna(), alternative='less')
fairness_diff_test_result = f"T-test results for Fairness Index groups - t-statistic: {t_stat_fairness_diff}, p-value: {p_value_fairness_diff}"

fairness_diff_test_result


'T-test results for Fairness Index groups - t-statistic: 8084.0, p-value: 0.028643071212242098'

In [22]:
# Hypothesis Test: Risk Aversion and Demand Responsiveness
# Null Hypothesis (H0): There is no significant difference in demand responsiveness based on risk aversion.
# Alternative Hypothesis (H1): There is a significant difference in demand responsiveness based on risk aversion.

# Calculate responsiveness to demand by taking the absolute difference between demand and stock for each group
high_risk_demand_response = abs(high_risk_averse['Demand'] - high_risk_averse['Stock'])
low_risk_demand_response = abs(low_risk_averse['Demand'] - low_risk_averse['Stock'])

# Conduct Mann-Whitney U test for demand responsiveness based on risk aversion
test_stat, p_value = mannwhitneyu(high_risk_demand_response.dropna(), low_risk_demand_response.dropna(), alternative='two-sided')

test_stat, p_value

"""
The Mann-Whitney U test yields a p-value of approximately 0.086, which does not indicate a statistically significant difference in demand responsiveness between 
high and low risk-averse participants. This suggests that risk aversion may not strongly impact how participants adjust their stock in response to demand changes.
"""


(89551.0, 0.08584266993168418)